# SRC and HCK Interaction Analysis Pipeline

### 1. Library Imports and Setup

In [ ]:
# Import all required libraries for the project
import pandas as pd

# Import custom PPI module
import importlib
import PPI_Script as ppi
importlib.reload(ppi)

<module 'PPI_Script' from '/home/gogllab/Desktop/Quantitative_Interactomics/PPI_Script.py'>

### 2. Data Loading

In [ ]:
# Read input files
df_Human= ppi.read_csv_file('/home/gogllab/Desktop/Quantitative_Interactomics/input/Human.csv', header=0)
df_Mouse= ppi.read_csv_file('/home/gogllab/Desktop/Quantitative_Interactomics/input/Mouse.csv', header=0)

# Get concentrations for SRC and HCK proteins in both species
Human_SRC_Concentration_µM = ppi.get_protein_concentration(df_Human, 'P12931')
Human_HCK_Concentration_µM = ppi.get_protein_concentration(df_Human, 'P08631')
Mouse_SRC_Concentration_µM = ppi.get_protein_concentration(df_Mouse, 'P05480')
Mouse_HCK_Concentration_µM = ppi.get_protein_concentration(df_Mouse, 'P08103')

# Print the concentration values
print("Human SRC Concentration (µM):", Human_SRC_Concentration_µM)
print("Human HCK Concentration (µM):", Human_HCK_Concentration_µM)     
print("Mouse SRC Concentration (µM):", Mouse_SRC_Concentration_µM)
print("Mouse HCK Concentration (µM):", Mouse_HCK_Concentration_µM)

### 3. Apparent Binding Constant (Kapp) Calculation

In [ ]:
# Calculate Kapp for SRC and HCK + delta_pKapp

df_Human = ppi.calculate_Kapp(df_Human, gene_id="HNRNPK")
df_Mouse = ppi.calculate_Kapp(df_Mouse, gene_id="HNRPK")

# Save results to CSV files
ppi.save_csv_file(df_Human, '/home/gogllab/Desktop/Quantitative_Interactomics/output/Human.csv')
ppi.save_csv_file(df_Mouse, '/home/gogllab/Desktop/Quantitative_Interactomics/output/Mouse.csv')

### 4. Complexome Analysis

In [ ]:
# Calculate human complexome

ppi.add_complexome_column(df_Human, Human_SRC_Concentration_μM, 'SRC')
ppi.add_complexome_column(df_Human, Human_HCK_Concentration_μM, 'HCK')
ppi.calculate_complexome_difference(df_Human, 'SRC', 'HCK')


# Calculate mouse complexome
ppi.add_complexome_column(df_Mouse, Mouse_SRC_Concentration_μM, 'SRC')
ppi.add_complexome_column(df_Mouse, Mouse_HCK_Concentration_μM, 'HCK')
ppi.calculate_complexome_difference(df_Mouse, 'SRC', 'HCK')


# Test: Display results for HNRNPK (Human) and HNRPK (Mouse)
for species, df, gene in [('Humain', df_Human, 'HNRNPK'), ('Souris', df_Mouse, 'HNRPK')]:
    row = df[df['Gene_ID'] == gene]
    print(f"\nTest sur {gene} ({species})\n", row[['Gene_ID', 'Uniprot_ID', 'Concentration_µM',
                                                    'SRC_pKapp_sign', 'SRC_Kapp_sign',
                                                    'HCK_pKapp_sign', 'HCK_Kapp_sign', 'delta_pKapp',
                                                    'SRC_Complexome', 'HCK_Complexome', 'delta_complexome']])
    
# Save updated CSV files
ppi.save_csv_file(df_Human,'/home/gogllab/Desktop/Quantitative_Interactomics/output/Human.csv')
ppi.save_csv_file(df_Mouse,'/home/gogllab/Desktop/Quantitative_Interactomics/output/Mouse.csv')


### 5. Log Scale Transformation

In [ ]:
# Apply log scale to selected columns
columns_to_log = ['Concentration_µM','SRC_Complexome', 'HCK_Complexome', 'delta_complexome']
df_Human = ppi.apply_log_transformation(df_Human, columns_to_log)
df_Mouse = ppi.apply_log_transformation(df_Mouse, columns_to_log)

# Test: Display transformed results for HNRNPK (Human) and HNRPK (Mouse)
for species, df, gene in [('Humain', df_Human, 'HNRNPK'), ('Souris', df_Mouse, 'HNRPK')]:
    row = df[df['Gene_ID'] == gene]
    print(f"\nTest sur {gene} ({species})\n", row.reindex(fill_value='N/A'))

# Save results to CSV files
ppi.save_csv_file(df_Human,'/home/gogllab/Desktop/Quantitative_Interactomics/output/Human.csv')
ppi.save_csv_file(df_Mouse,'/home/gogllab/Desktop/Quantitative_Interactomics/output/Mouse.csv')

### 6. Intra-species Complexome Ratio Calculation

In [ ]:
# Calculate intra-species complexome ratio
df_Human = ppi.calculate_intra_complexome_ratio(df_Human, 'SRC', 'HCK')
df_Mouse = ppi.calculate_intra_complexome_ratio(df_Mouse, 'SRC', 'HCK')

# Test: Display results for HNRNPK (Human) and HNRPK (Mouse)
for species, df, gene in [('Humain', df_Human, 'HNRNPK'), ('Souris', df_Mouse, 'HNRPK')]:
    row = df[df['Gene_ID'] == gene]
    print(f"\nTest sur {gene} ({species})\n", row.reindex(fill_value='N/A'))

# Save results to CSV files
ppi.save_csv_file(df_Human,'/home/gogllab/Desktop/Quantitative_Interactomics/output/Human.csv')
ppi.save_csv_file(df_Mouse,'/home/gogllab/Desktop/Quantitative_Interactomics/output/Mouse.csv')


### 7. Creating Matched Dataset

In [ ]:
# Create matching file between human and mouse data

ppi.create_merged_species_dataset(
    human_path="/home/gogllab/Desktop/Quantitative_Interactomics/output/Human.csv",
    mouse_path="/home/gogllab/Desktop/Quantitative_Interactomics/output/Mouse.csv",
    output_path="/home/gogllab/Desktop/Quantitative_Interactomics/output/Matching.csv",
)

### 8. Venn Diagram Analysis of Partners

In [ ]:
# Reload the matched dataset
df_Matching = pd.read_csv('/home/gogllab/Desktop/Quantitative_Interactomics/output/Matching.csv', header=[0, 1])

# Generate Venn diagrams for different comparisons

# Intra-species comparisons (SRC vs HCK partners)
ppi.plot_venn_partners(df_Matching, 'Human_Venn_Diagram', species='Human', interspecies=False)
ppi.plot_venn_partners(df_Matching, 'Mouse_Venn_Diagram', species='Mouse', interspecies=False)

# Inter-species comparisons (Human vs Mouse partners for each protein)
ppi.plot_venn_partners(df_Matching, 'SRC_H_M_Venn_Diagram', bait='SRC', interspecies=True)
ppi.plot_venn_partners(df_Matching, 'HCK_H_M_Venn_Diagram', bait='HCK', interspecies=True)

### 9. Complexome Visualization

In [ ]:
# Load the matched dataset
df_Matching = pd.read_csv('/home/gogllab/Desktop/Quantitative_Interactomics/output/Matching.csv', header=[0, 1])

# Plot complexome for each combination of species and bait
ppi.plot_complexome_analysis(df_Matching, "Human", "SRC", "SRC_pKapp_sign", "log_SRC_Complexome")
ppi.plot_complexome_analysis(df_Matching, "Human", "HCK", "HCK_pKapp_sign", "log_HCK_Complexome")
ppi.plot_complexome_analysis(df_Matching, "Mouse", "SRC", "SRC_pKapp_sign", "log_SRC_Complexome")
ppi.plot_complexome_analysis(df_Matching, "Mouse", "HCK", "HCK_pKapp_sign", "log_HCK_Complexome")


### 10. Comparative Analysis and Plotting

In [ ]:
# Path to the matched dataset
csv_file = '/home/gogllab/Desktop/Quantitative_Interactomics/output/Matching.csv'

# Generate various comparative plots
# Intra-species comparisons of SRC vs. HCK
ppi.analyze_and_plot_correlation(csv_file, "Human_SRC_HCK",  "Human", 'HCK_pKapp_sign', 'Human',  'SRC_pKapp_sign', 'Human SRC vs HCK', 'affinity', annotate_genes=False)
ppi.analyze_and_plot_correlation(csv_file, "Mouse_SRC_HCK",  "Mouse", 'HCK_pKapp_sign', 'Mouse',  'SRC_pKapp_sign', 'Mouse SRC vs HCK', 'affinity', annotate_genes=False)

# Inter-species comparisons for SRC and HCK
ppi.analyze_and_plot_correlation(csv_file, "Human_Mouse_SRC",  "Mouse", 'SRC_pKapp_sign', 'Human',  'SRC_pKapp_sign', 'Human vs Mouse SRC', 'affinity', annotate_genes=False)
ppi.analyze_and_plot_correlation(csv_file, "Human_Mouse_HCK",  "Mouse", 'HCK_pKapp_sign', 'Human',  'HCK_pKapp_sign', 'Human  vs Mouse HCK', 'affinity', annotate_genes=False)
ppi.analyze_and_plot_correlation(csv_file, 'Human_vs_Mouse_pKapp', "Mouse", 'delta_pKapp', "Human", 'delta_pKapp', 'Human vs Mouse', 'delta_pKapp', annotate_genes=False)

# Complexome comparisons
ppi.analyze_and_plot_correlation(csv_file, "Human_Complexome_SRC_vs_HCK",  "Human", 'log_HCK_Complexome', 'Human',  'log_SRC_Complexome', ' Human Complexome SRC VS HCK', 'log_complexome', annotate_genes=False)
ppi.analyze_and_plot_correlation(csv_file, "Mouse_Complexome_SRC_vs_HCK",  "Mouse", 'log_HCK_Complexome', 'Mouse',  'log_SRC_Complexome', 'Mouse Complexome SRC VS HCK', 'log_complexome', annotate_genes=False)
ppi.analyze_and_plot_correlation(csv_file, 'Human_vs_Mouse_Complexome', "Mouse",'ratio_log_intra', "Human", 'ratio_log_intra', 'Human vs Mouse Complexome', 'log_ratio_complexome',annotate_genes=False)

